In [2]:
import mysql.connector
import pandas as pd
import numpy as np
import sys
import os
import itertools
from sentence_transformers import SentenceTransformer
import torch


# MySQL 서버에 연결
conn = mysql.connector.connect(
    host='127.0.0.1',      # 호스트 이름
    user='root',       # MySQL 사용자 이름
    password='hj010701',   # MySQL 사용자 비밀번호
    database='employee'  # 연결할 데이터베이스 이름
)

# 커서 생성
cursor = conn.cursor()
sql_query = "SELECT * FROM member_assign_50to100"
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

member_based_okr_assignments = pd.DataFrame(result, columns=column_names)

################################################################################
sql_query = "SELECT * FROM okr_30to60"
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

okr_df = pd.DataFrame(result, columns=column_names)


################################################################################
sql_query = '''
SELECT *
FROM member_assign_50to100
JOIN okr_30to60 
ON okr_30to60.OKR_NUM IN (member_assign_50to100.project1, member_assign_50to100.project2, member_assign_50to100.project3);
'''
cursor.execute(sql_query)

result = cursor.fetchall()

column_names = [i[0] for i in cursor.description]

member_okr = pd.DataFrame(result, columns=column_names)



# CSV 파일 불러오기
df = member_okr

# 'Member' 컬럼 기준으로 오름차순 정렬
df_sorted = df.sort_values(by='Member', ascending=True)

# 데이터를 저장할 리스트 초기화
flattened_data = []

# 전체 열 개수 확인
num_columns = df_sorted.shape[1]

# 데이터 처리 및 열 범위에 따른 조건 설정
for i in range(50):
    # 열 범위 설정
    if 0 <= i < 10 or 50<= i <60:
        selected_columns = [12] + [col for col in range(14, 20) if col < num_columns]
    elif 10 <= i < 20 or 60<= i <70:
        selected_columns = [12] + [col for col in range(20, 26) if col < num_columns]
    elif 20 <= i < 30 or 70<= i <80:
        selected_columns = [12] + [col for col in range(26, 32) if col < num_columns]
    elif 30 <= i < 40 or 80<= i <90:
        selected_columns = [12] + [col for col in range(32, 38) if col < num_columns]
    elif 40 <= i < 50 or 90<= i <100:
        selected_columns = [12] + [col for col in range(38, 44) if col < num_columns]

    # 선택된 열의 첫 번째 데이터 가져오기
    first_row_data = df_sorted.iloc[3 * i : 3 * (i + 1), selected_columns[0]].T.tolist()

    # 나머지 열 데이터 가져와 병합
    other_data = df_sorted.iloc[3 * i : 3 * (i + 1), selected_columns[1:]].values.flatten().tolist()

    # 데이터 조합
    combined_data = [i + 1, np.nan] + first_row_data + other_data
    flattened_data.append(combined_data)

# 컬럼명 설정 (최대 길이에 맞게 조정)
column_names = [
    'member', 'N_OKR', 'pr1_score', 'pr2_score', 'pr3_score', 
    'pr1_1', 'pr1_2', 'pr1_3', 'pr1_4', 'pr1_5', 'pr1_6', 
    'pr2_1', 'pr2_2', 'pr2_3', 'pr2_4', 'pr2_5', 'pr2_6', 
    'pr3_1', 'pr3_2', 'pr3_3', 'pr3_4', 'pr3_5', 'pr3_6'
]
max_length = max(len(row) for row in flattened_data)
adjusted_column_names = column_names[:max_length]

# 새로운 DataFrame 생성 및 CSV로 저장
data = pd.DataFrame(flattened_data, columns=adjusted_column_names)

data=data.iloc[:,1:]


objectives = okr_df['Objective']
posted_OKR=okr_df['OKR_NUM']

posted_OKR=posted_OKR.str.replace("OKR_", "").astype(float)
score=okr_df['Objective Score']

def generate_combinations_3d(data, num_parts=5):
    # Convert data to numpy array for easier manipulation
    data_values = data.values
    
    # Split data into equal parts
    part_size = len(data_values) // num_parts
    parts = [data_values[i * part_size:(i + 1) * part_size] for i in range(num_parts)]
    
    # Generate all possible combinations from the parts (row-wise combinations)
    combinations = list(itertools.product(*parts))
    
    # Convert combinations to a 3D numpy array
    combinations_3d = np.array(combinations)
    
    return combinations_3d



model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def get_similarities(n_okr, objectives):
    # n_okr와 각 objective에 대해 임베딩 생성
    n_okr_embedding = model.encode(n_okr, convert_to_tensor=True)
    similarities = []

    for objective in objectives:
        obj_embedding = model.encode(objective, convert_to_tensor=True)
        similarity = torch.nn.functional.cosine_similarity(n_okr_embedding, obj_embedding, dim=-1).item()
        similarities.append(similarity)

    return similarities

def calculate_weighted_scores(n_okr): 
    
    df = member_okr

    # 각 멤버의 유사도 계산 결과를 저장할 리스트
    weighted_sums = []

    # 각 멤버별로 데이터를 그룹화
    grouped_df = df.groupby('Member')

    for member, group in grouped_df:
        objectives = group['Objective'].tolist()
        objective_scores = group['Objective Score'].tolist()

        # 유사도 계산 (멤버별 3개의 okr에 대한 유사도)
        similarities = get_similarities(n_okr, objectives)
        
        #print(similarities)

        total_weighted_score = 0
        valid_count = 0

        # 유사도와 Objective Score를 곱해 가중합 계산
        for similarity, objective_score, objective in zip(similarities, objective_scores, objectives):
            if objective != n_okr:  # objective가 같은 경우 제외
                weighted_score = similarity * objective_score
                total_weighted_score += weighted_score
                valid_count += 1

        if valid_count > 0:
            weighted_sums.append((member, total_weighted_score / valid_count))
        else:
            weighted_sums.append((member, 0))  # 모든 유사도가 1에 가까운 경우

    return weighted_sums

all_data_f = []

# 사용자가 입력한 n_okr, posted, label 값
n_okr_input = "Your custom OKR here"  # 여기에 입력하고 싶은 OKR 문장을 넣으세요
posted_input = 61.0  # OKR 번호
label_input = np.nan

# 루프: 기존 30개의 OKR 자리를 사용자가 입력한 n_okr로 대체
for _ in range(len(objectives)):  # objectives의 길이를 기준으로 반복
    # Step 2: calculate_weighted_scores 함수 사용하여 weighted_sums 계산
    weighted_sums = calculate_weighted_scores(n_okr_input)[:50]  # 멤버 수는 최대 50개로 제한

    # Step 3: weighted_sums에서 두 번째 값을 추출
    weighted_values = [value[1] for value in weighted_sums]

    # Step 4: data의 첫 번째 열(0열)에 weighted_values 추가
    weighted_array = np.array(weighted_values)

    if data.shape[0] == len(weighted_values):
        data.iloc[:, 0] = weighted_array  # pandas의 iloc 사용하여 첫 번째 열에 할당
        data["member"] = data.index.astype(int)
        data["posted"] = posted_input  # 사용자 입력값으로 업데이트
        data['label'] = label_input  # 사용자 입력값으로 업데이트
    else:
        print(f"샘플 수가 일치하지 않습니다. data 행 수: {data.shape[0]}, weighted_values 길이: {len(weighted_values)}")
    
    # Generate combinations based on the updated data
    data_3d = generate_combinations_3d(data.iloc[:, :], num_parts=5)

    # data_f 계산
    data_f = np.concatenate((data_3d[:, :, 0:1], data_3d[:, :, 4:]), axis=2)

    # data_f를 dim=0에서 쌓기 위해 리스트에 저장
    all_data_f.append(data_f)

# Step 5: dim=0에서 모든 data_f 연결
final_data_f = np.concatenate(all_data_f, axis=0)


c:\Users\82102\miniconda3\envs\hyunjae\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import torch
from algorithm import TeamTransformer  # TeamTransformer 클래스 불러오기

# 모델 파라미터 설정 (학습 시와 동일해야 함)
embedding_dim = 19
seq_len = 5
output_dim = 1
n_heads = 1
n_layers = 3
hidden_dim = 64
dropout_rate = 0.2

# 모델 초기화
model = TeamTransformer(
    embedding_dim=embedding_dim,
    n_heads=n_heads,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    output_dim=output_dim,
    dropout_rate=dropout_rate,
)

# 저장된 가중치 로드
state_dict = torch.load('best_model_weights.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# 평가 모드로 전환
model.eval()

# 예측 수행
input_tensor = torch.tensor(final_data_f, dtype=torch.float32)  # 입력 데이터 변환
with torch.no_grad():
    predictions, _ = model(input_tensor)  # 모델 예측 수행

predicted_labels = predictions.numpy()
final_data_f[:, :, -1] = predicted_labels
final_data_f

C:\Users\82102\AppData\Local\Temp\ipykernel_19272\495916134.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('best_model_weights.pth', map_locati

RuntimeError: Error(s) in loading state_dict for TeamTransformer:
	size mismatch for transformer_encoder.layers.0.self_attn.in_proj_weight: copying a param with shape torch.Size([57, 19]) from checkpoint, the shape in current model is torch.Size([66, 22]).
	size mismatch for transformer_encoder.layers.0.self_attn.in_proj_bias: copying a param with shape torch.Size([57]) from checkpoint, the shape in current model is torch.Size([66]).
	size mismatch for transformer_encoder.layers.0.self_attn.out_proj.weight: copying a param with shape torch.Size([19, 19]) from checkpoint, the shape in current model is torch.Size([22, 22]).
	size mismatch for transformer_encoder.layers.0.self_attn.out_proj.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([22]).
	size mismatch for transformer_encoder.layers.0.linear1.weight: copying a param with shape torch.Size([64, 19]) from checkpoint, the shape in current model is torch.Size([64, 22]).
	size m

In [6]:
# 학습된 모델 불러오기
trained_model = torch.load('best_model_weights.pth', map_location=torch.device('cpu'))  # 모델 경로를 지정하세요
trained_model.eval()  # 평가 모드로 전환

# final_data_f에서 예측 수행
input_tensor = torch.tensor(final_data_f, dtype=torch.float32)  # 모델 입력 형태로 변환

# 예측 수행
with torch.no_grad():
    predictions = trained_model(input_tensor)  # 모델 예측
    predicted_labels = torch.argmax(predictions, dim=-1).numpy()  # 예측된 라벨 추출

# 예측된 라벨로 마지막 열(22번째)을 교체
final_data_f[:, :, -1] = predicted_labels[:, None]  # 마지막 열을 라벨로 교체

# 결과 확인
print("22번째 열(마지막 열)이 라벨로 교체된 데이터셋:\n", final_data_f)



C:\Users\82102\AppData\Local\Temp\ipykernel_8232\2319489034.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_model = torch.load('best_model_weights.pth', map_loca

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [2]:
final_data_f.shape

(3000000, 5, 22)

In [3]:
final_data_f

array([[[ 7.93741633,  1.        ,  3.        , ...,  0.        ,
         61.        ,         nan],
        [11.63435521,  5.        ,  3.        , ..., 10.        ,
         61.        ,         nan],
        [ 7.1600523 ,  5.        ,  4.        , ..., 20.        ,
         61.        ,         nan],
        [ 9.45510848,  3.        ,  1.        , ..., 30.        ,
         61.        ,         nan],
        [ 4.64377764,  1.        ,  2.        , ..., 40.        ,
         61.        ,         nan]],

       [[ 7.93741633,  1.        ,  3.        , ...,  0.        ,
         61.        ,         nan],
        [11.63435521,  5.        ,  3.        , ..., 10.        ,
         61.        ,         nan],
        [ 7.1600523 ,  5.        ,  4.        , ..., 20.        ,
         61.        ,         nan],
        [ 9.45510848,  3.        ,  1.        , ..., 30.        ,
         61.        ,         nan],
        [ 5.31392879,  2.        ,  3.        , ..., 41.        ,
         61.  